# Capstone project Title

## Project description 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## Download and Explore Dataset

Description of the dataset and the source 

In [2]:
df_vancouver=pd.read_csv('vancouver_dataset.csv')
df_vancouver.rename(columns={'MAPID': 'PostalCode', 'Name':'Neighborhood'},inplace=True)
df_vancouver

,PostalCode,Neighborhood,Latitude,Longitude
0,AR,Arbutus-Ridge,49.246805,-123.161669
1,CBD,Downtown,49.280747,-123.116567
2,FAIR,Fairview,49.264540,-123.131049
3,GW,Grandview-Woodland,49.276440,-123.066728
4,HS,Hastings-Sunrise,49.277934,-123.040270
5,MARP,Marpole,49.210207,-123.128382
6,RP,Riley Park,49.244766,-123.103147
7,SHAU,Shaughnessy,49.245681,-123.139760
8,STR,Strathcona,49.278220,-123.088235
9,WE,West End,49.285011,-123.135438


## Create a map of Vancouver with neighborhoods superimposed on top.

In [3]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver, BC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver, BC are 49.2608724, -123.1139529.


In [4]:
# create map of  Vancouver using latitude and longitude values
map_vancouver = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,neighborhood in zip(df_vancouver['Latitude'], df_vancouver['Longitude'],df_vancouver['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver)  
    
map_vancouver

## Exploring neighborhoods in Vancouver with the Foursquare API

### Defining credentials and version for Foursquare

In [5]:
CLIENT_ID = 'XNE1FN0GRLSRNRWJGKNJOGMV3CGXVQO525RVF1CGAIGBXFRI' # your Foursquare ID
CLIENT_SECRET = 'WAZCUGEY4E04HMLKSMISSD55VLLRVU4M5YATOG4IXONL4FRE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XNE1FN0GRLSRNRWJGKNJOGMV3CGXVQO525RVF1CGAIGBXFRI
CLIENT_SECRET:WAZCUGEY4E04HMLKSMISSD55VLLRVU4M5YATOG4IXONL4FRE


Now, let's get the top 100 venues that are in Vancouver Downtown neighborhood 

In [6]:
neighborhood_latitude = df_vancouver.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_vancouver.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_vancouver.loc[1, 'Neighborhood'] # neighborhood name

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=XNE1FN0GRLSRNRWJGKNJOGMV3CGXVQO525RVF1CGAIGBXFRI&client_secret=WAZCUGEY4E04HMLKSMISSD55VLLRVU4M5YATOG4IXONL4FRE&v=20180605&ll=49.28074707,-123.11656699999999&radius=500&limit=100'

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#recieving the reqsults from the Foursquae API

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head() #show the first 5 venues 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,name,categories,lat,lng
0,L'Hermitage,Hotel,49.280139,-123.117480
1,Gotham Steakhouse & Cocktail Bar,Steakhouse,49.282830,-123.115865
2,Medina Café,Breakfast Spot,49.280565,-123.116859
3,JJ Bean,Coffee Shop,49.279382,-123.115181
4,Paramount Fine Foods,Lebanese Restaurant,49.280452,-123.118586


In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

54 venues were returned by Foursquare.


## Explore venues in Vancouver 

describe

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
vancouver_venues = getNearbyVenues(names=df_vancouver['Neighborhood'],
                                   latitudes=df_vancouver['Latitude'],
                                   longitudes=df_vancouver['Longitude']
                                  )
vancouver_venues.head()
vancouver_venues.shape


Arbutus-Ridge
Downtown
Fairview
Grandview-Woodland
Hastings-Sunrise
Marpole
Riley Park
Shaughnessy
Strathcona
West End
Dunbar-Southlands
Kerrisdale
Killarney
Kitsilano
South Cambie
Victoria-Fraserview
Kensington-Cedar Cottage
Mount Pleasant
Oakridge
Renfrew-Collingwood
Sunset
West Point Grey


(453, 7)

In [12]:
vancouver_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbutus-Ridge,49.246805,-123.161669,Arbutus Village Park,49.249486,-123.158716,Park
1,Arbutus-Ridge,49.246805,-123.161669,Total Corporate Learning Inc.,49.242441,-123.161609,Business Service
2,Arbutus-Ridge,49.246805,-123.161669,Triangle Park,49.245061,-123.167914,Park
3,Downtown,49.280747,-123.116567,L'Hermitage,49.280139,-123.117480,Hotel
4,Downtown,49.280747,-123.116567,Gotham Steakhouse & Cocktail Bar,49.282830,-123.115865,Steakhouse


In [13]:
vancouver_venues.shape

(453, 7)

In [14]:
vancouver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arbutus-Ridge,3,3,3,3,3,3
Downtown,54,54,54,54,54,54
Dunbar-Southlands,6,6,6,6,6,6
Fairview,26,26,26,26,26,26
Grandview-Woodland,37,37,37,37,37,37
Hastings-Sunrise,14,14,14,14,14,14
Kensington-Cedar Cottage,16,16,16,16,16,16
Kerrisdale,4,4,4,4,4,4
Killarney,19,19,19,19,19,19


Unique categories number

In [15]:
print('There are {} uniques categories.'.format(len(vancouver_venues['Venue Category'].unique())))

There are 154 uniques categories.


## Analyze Each Neighborhood

In [16]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot['Neighborhood'] = vancouver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()


,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Belgian Restaurant,Board Shop,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Optical Shop,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Skating Rink,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Downtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Downtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
vancouver_onehot.shape

(453, 155)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
vancouver_grouped = vancouver_onehot.groupby('Neighborhood').mean().reset_index()
vancouver_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Belgian Restaurant,Board Shop,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Optical Shop,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Skating Rink,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Arbutus-Ridge,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.018519,0.000000,0.000000,0.018519,0.018519,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.018519,0.00,0.018519,0.000000,0.055556,0.037037,0.000000,0.018519,0.000000,0.018519,0.000000,0.018519,0.000000

In [19]:
vancouver_grouped.shape

(21, 155)

In [20]:
num_top_venues = 5

for hood in vancouver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arbutus-Ridge----
               venue  freq
0               Park  0.67
1   Business Service  0.33
2  Outdoor Sculpture  0.00
3        Music Venue  0.00
4         Nail Salon  0.00


----Downtown----
                           venue  freq
0                          Hotel  0.11
1                     Restaurant  0.06
2                    Coffee Shop  0.06
3                            Bar  0.06
4  Vegetarian / Vegan Restaurant  0.04


----Dunbar-Southlands----
                 venue  freq
0        Grocery Store  0.33
1          Coffee Shop  0.17
2         Liquor Store  0.17
3             Bus Stop  0.17
4  Japanese Restaurant  0.17


----Fairview----
                 venue  freq
0          Coffee Shop  0.12
1                 Park  0.08
2  Japanese Restaurant  0.08
3     Asian Restaurant  0.08
4       Breakfast Spot  0.08


----Grandview-Woodland----
               venue  freq
0        Coffee Shop  0.11
1        Pizza Place  0.08
2      Deli / Bodega  0.05
3            Theater  0.05
4  I

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = vancouver_grouped['Neighborhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arbutus-Ridge,Park,Business Service,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,Downtown,Hotel,Bar,Coffee Shop,Restaurant,Sandwich Place,Steakhouse,Seafood Restaurant,Concert Hall,Vegetarian / Vegan Restaurant,Art Gallery
2,Dunbar-Southlands,Grocery Store,Bus Stop,Coffee Shop,Japanese Restaurant,Liquor Store,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Fairview,Coffee Shop,Park,Asian Restaurant,Breakfast Spot,Japanese Restaurant,Gym,Salon / Barbershop,Camera Store,Korean Restaurant,Spa
4,Grandview-Woodland,Coffee Shop,Pizza Place,Theater,Café,Deli / Bodega,Vegetarian / Vegan Restaurant,Indian Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop


## Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 5

vancouver_grouped_clustering = vancouver_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

vancouver_merged = df_vancouver

# merge vancouver_grouped with vancouver_data to add latitude/longitude for each neighborhood
vancouver_merged = vancouver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

vancouver_merged # check the last columns!


,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AR,Arbutus-Ridge,49.246805,-123.161669,2.0,Park,Business Service,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,CBD,Downtown,49.280747,-123.116567,1.0,Hotel,Bar,Coffee Shop,Restaurant,Sandwich Place,Steakhouse,Seafood Restaurant,Concert Hall,Vegetarian / Vegan Restaurant,Art Gallery
2,FAIR,Fairview,49.264540,-123.131049,1.0,Coffee Shop,Park,Asian Restaurant,Breakfast Spot,Japanese Restaurant,Gym,Salon / Barbershop,Camera Store,Korean Restaurant,Spa
3,GW,Grandview-Woodland,49.276440,-123.066728,1.0,Coffee Shop,Pizza Place,Theater,Café,Deli / Bodega,Vegetarian / Vegan Restaurant,Indian Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop
4,HS,Hastings-Sunrise,49.277934,-123.040270,1.0,Bus Station,Theme Park Ride / Attraction,Café,Event Space,Bridal Shop,Portuguese Restaurant,Office,Gas Station,BBQ Joint,Pizza Place
5,MARP,Marpole,49.210207,-123.128382,0.0,Bus Stop,Plaza,Pizza Place,Bus Station,Japanese Restaurant,Taiwanese Restaurant,Electronics Store,Farmers Market,Falafel Restaurant,Event Space
6,RP,Riley Park,49.244766,-123.103147,1.0,Vietnamese Restaurant,Coffee Shop,Farmers Market,Japanese Restaurant,Chinese Restaurant,Café,Skating Rink,Lounge,Restaurant,Sporting Goods Shop
7,SHAU,Shaughnessy,49.245681,-123.139760,3.0,Bus Stop,Park,Print Shop,Chocolate Shop,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
8,STR,Strathcona,49.278220,-123.088235,1.0,Park,Coffee Shop,Restaurant,Deli / Bodega,Brewery,Cheese Shop,Pub,Food Truck,Sandwich Place,Falafel Restaurant
9,WE,West End,49.285011,-123.135438,1.0,Café,Sushi Restaurant,Gay Bar,Farmers Market,Hotel,Coffee Shop,Noodle House,Pub,Restaurant,Falafel Restaurant


In [25]:
# delete all NaN values

vancouver_merged1=vancouver_merged.dropna()
vancouver_merged1



,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AR,Arbutus-Ridge,49.246805,-123.161669,2.0,Park,Business Service,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,CBD,Downtown,49.280747,-123.116567,1.0,Hotel,Bar,Coffee Shop,Restaurant,Sandwich Place,Steakhouse,Seafood Restaurant,Concert Hall,Vegetarian / Vegan Restaurant,Art Gallery
2,FAIR,Fairview,49.264540,-123.131049,1.0,Coffee Shop,Park,Asian Restaurant,Breakfast Spot,Japanese Restaurant,Gym,Salon / Barbershop,Camera Store,Korean Restaurant,Spa
3,GW,Grandview-Woodland,49.276440,-123.066728,1.0,Coffee Shop,Pizza Place,Theater,Café,Deli / Bodega,Vegetarian / Vegan Restaurant,Indian Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop
4,HS,Hastings-Sunrise,49.277934,-123.040270,1.0,Bus Station,Theme Park Ride / Attraction,Café,Event Space,Bridal Shop,Portuguese Restaurant,Office,Gas Station,BBQ Joint,Pizza Place
5,MARP,Marpole,49.210207,-123.128382,0.0,Bus Stop,Plaza,Pizza Place,Bus Station,Japanese Restaurant,Taiwanese Restaurant,Electronics Store,Farmers Market,Falafel Restaurant,Event Space
6,RP,Riley Park,49.244766,-123.103147,1.0,Vietnamese Restaurant,Coffee Shop,Farmers Market,Japanese Restaurant,Chinese Restaurant,Café,Skating Rink,Lounge,Restaurant,Sporting Goods Shop
7,SHAU,Shaughnessy,49.245681,-123.139760,3.0,Bus Stop,Park,Print Shop,Chocolate Shop,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
8,STR,Strathcona,49.278220,-123.088235,1.0,Park,Coffee Shop,Restaurant,Deli / Bodega,Brewery,Cheese Shop,Pub,Food Truck,Sandwich Place,Falafel Restaurant
9,WE,West End,49.285011,-123.135438,1.0,Café,Sushi Restaurant,Gay Bar,Farmers Market,Hotel,Coffee Shop,Noodle House,Pub,Restaurant,Falafel Restaurant


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged1['Latitude'], vancouver_merged1['Longitude'], vancouver_merged1['Neighborhood'], vancouver_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine clusters

In [27]:
vancouver_merged1.loc[vancouver_merged1['Cluster Labels'] == 0, vancouver_merged1.columns[[1] + list(range(5, vancouver_merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Marpole,Bus Stop,Plaza,Pizza Place,Bus Station,Japanese Restaurant,Taiwanese Restaurant,Electronics Store,Farmers Market,Falafel Restaurant,Event Space
19,Renfrew-Collingwood,Convenience Store,Bus Station,Bus Stop,Park,Pizza Place,Yoga Studio,Ethiopian Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [28]:
vancouver_merged1.loc[vancouver_merged1['Cluster Labels'] == 1, vancouver_merged1.columns[[1] + list(range(5, vancouver_merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown,Hotel,Bar,Coffee Shop,Restaurant,Sandwich Place,Steakhouse,Seafood Restaurant,Concert Hall,Vegetarian / Vegan Restaurant,Art Gallery
2,Fairview,Coffee Shop,Park,Asian Restaurant,Breakfast Spot,Japanese Restaurant,Gym,Salon / Barbershop,Camera Store,Korean Restaurant,Spa
3,Grandview-Woodland,Coffee Shop,Pizza Place,Theater,Café,Deli / Bodega,Vegetarian / Vegan Restaurant,Indian Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop
4,Hastings-Sunrise,Bus Station,Theme Park Ride / Attraction,Café,Event Space,Bridal Shop,Portuguese Restaurant,Office,Gas Station,BBQ Joint,Pizza Place
6,Riley Park,Vietnamese Restaurant,Coffee Shop,Farmers Market,Japanese Restaurant,Chinese Restaurant,Café,Skating Rink,Lounge,Restaurant,Sporting Goods Shop
8,Strathcona,Park,Coffee Shop,Restaurant,Deli / Bodega,Brewery,Cheese Shop,Pub,Food Truck,Sandwich Place,Falafel Restaurant
9,West End,Café,Sushi Restaurant,Gay Bar,Farmers Market,Hotel,Coffee Shop,Noodle House,Pub,Restaurant,Falafel Restaurant
11,Kerrisdale,Bar,Pool,Park,Café,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
12,Killarney,Bus Stop,Fast Food Restaurant,Gas Station,Farmers Market,Shopping Mall,Recreation Center,Sushi Restaurant,Salon / Barbershop,Chinese Restaurant,Juice Bar
13,Kitsilano,Coffee Shop,Japanese Restaurant,Food Truck,Optical Shop,Wine Shop,Grocery Store,Pizza Place,Bakery,Bank,Toy / Game Store


In [29]:
vancouver_merged1.loc[vancouver_merged1['Cluster Labels'] == 2, vancouver_merged1.columns[[1] + list(range(5, vancouver_merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arbutus-Ridge,Park,Business Service,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [30]:
vancouver_merged1.loc[vancouver_merged1['Cluster Labels'] == 3, vancouver_merged1.columns[[1] + list(range(5, vancouver_merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Shaughnessy,Bus Stop,Park,Print Shop,Chocolate Shop,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [31]:
vancouver_merged1.loc[vancouver_merged1['Cluster Labels'] == 4, vancouver_merged1.columns[[1] + list(range(5, vancouver_merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Dunbar-Southlands,Grocery Store,Bus Stop,Coffee Shop,Japanese Restaurant,Liquor Store,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
